# Testing

* Libraries

In [24]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import mediapipe as mp


* Model Loading

In [25]:
model = load_model("model_save/Latest_lstm_action_recognition_model.keras")

* Define Paths and Actions

In [26]:
# Define paths and actions
data_path = "action_data"  # Main directory for data
actions = ["Walking", "Squat", "Crunching", "Alternate Toe Touch", "Jumping Jacks", "Right Shot","Left Shot"]  # Actions to collect data for
threshold = 0.8  # Confidence threshold for action recognition
motion_threshold = 0.2  # Threshold for minimum motion to classify as a valid action
required_sequences = 30  # Expected number of sequences per action
sequence_length = 20  # Length of each sequence

* Prediction

In [27]:
# Mediapipe setup
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Define actions

threshold = 0.8  # Confidence threshold for action recognition
motion_threshold = 0.2  # Threshold for minimum motion to classify as a valid action
sequence_length = 20  # Length of sequence for the model

# Extract keypoints function
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    right_leg = np.array([[res.x, res.y, res.z] for res in [results.pose_landmarks.landmark[i] for i in [23, 25, 27, 29, 31]]]).flatten() if results.pose_landmarks else np.zeros(5 * 3)
    left_leg = np.array([[res.x, res.y, res.z] for res in [results.pose_landmarks.landmark[i] for i in [24, 26, 28, 30, 32]]]).flatten() if results.pose_landmarks else np.zeros(5 * 3)
    return np.concatenate([pose, lh, rh, right_leg, left_leg ])


# Start webcam feed
cap = cv2.VideoCapture(0)
holistic = mp_holistic.Holistic()
# Set the resolution to 1080p
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)  # Set the frame width to 1920 (1080p)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
sequence = []  # Stores the current sequence of keypoints
prev_keypoints = None  # Stores keypoints from the previous frame

print("Press 'q' to exit.")

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = holistic.process(frame_rgb)
    
    # Draw landmarks on the frame
    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

    # Extract keypoints
    keypoints = extract_keypoints(results)
    sequence.append(keypoints)
    sequence = sequence[-sequence_length:]  # Maintain only the last 'sequence_length' frames

    # Motion detection: Compare keypoints with the previous frame
    if prev_keypoints is not None:
        motion = np.linalg.norm(keypoints - prev_keypoints)
    else:
        motion = 0  # No motion in the first frame

    prev_keypoints = keypoints  # Update the previous keypoints for the next iteration

    # If motion is below the threshold, classify as "Unknown"
    if motion < motion_threshold:
        action = "Unknown"
        confidence = 0  # No confidence when motion is too low
    else:
        # Make predictions only if the sequence is complete
        if len(sequence) == sequence_length:
            standardized_sequence = np.array(sequence)

            # Debugging: Check the sequence shape
            print(f"Testing sequence shape: {standardized_sequence.shape}")

            # Ensure the sequence matches the model's expected shape
            if standardized_sequence.shape[1] != 255:  # Update 225 if your feature size is different
                raise ValueError(f"Expected feature size 255, but got {standardized_sequence.shape[1]}")

            standardized_sequence = standardized_sequence.reshape((1, sequence_length, 255))  # Reshape to match input shape
            predictions = model.predict(standardized_sequence)[0]
            action_index = np.argmax(predictions)
            confidence = predictions[action_index]

            if confidence > threshold:
                action = actions[action_index]
            else:
                action = "Unknown"
        else:
            action = "Unknown"

    # Overlay all action names on the frame with dynamic font color and size
    y_position = 50  # Initial Y position for displaying actions
    for act in actions:
        if act == action:
            font_color = (0, 0, 255)  # Red for the predicted action
            font_scale = 1.3  # Larger font size for the predicted action
        else:
            font_color = (255, 0, 0)  # Blue for unpredicted actions
            font_scale = 1.2  # Default font size for unpredicted actions
        cv2.putText(frame, act, (10, y_position), cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_color, 2)
        y_position += 50  # Increment Y position for the next action

    # Overlay motion details on the frame
    cv2.putText(frame, f"Motion: {motion:.2f}", (10, y_position + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Display the frame
    cv2.imshow("Action Recognition", frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):  # Exit on pressing 'q'
        break

cap.release()
cv2.destroyAllWindows()
holistic.close()

Press 'q' to exit.
Testing sequence shape: (20, 255)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step
Testing sequence shape: (20, 255)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Testing sequence shape: (20, 255)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Testing sequence shape: (20, 255)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Testing sequence shape: (20, 255)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Testing sequence shape: (20, 255)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Testing sequence shape: (20, 255)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Testing sequence shape: (20, 255)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Testing sequence shape: (20, 255)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Testing sequence shape: (20, 255)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Testing sequence shape: (20, 255)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Testing sequence shape: (20, 255)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Testing sequence shape: (20, 255)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Testing sequence shape: (20, 255)
1/1 ━━━━━━